In [1]:
import pickle
import pandas as pd
import torch
from torch import nn
from sklearn.preprocessing import LabelEncoder
from concurrent.futures import ProcessPoolExecutor
import numpy as np
from operator import itemgetter
THREADS = 16
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
import time
import torch.nn.functional as F
loss_func = torch.nn.MSELoss()

In [2]:
ratings = pd.read_pickle('MOVIELENS/ml-latest-small/small_ratings_movielens.pkl')
films = pd.read_pickle('MOVIELENS/ml-latest-small/small_films_movielens.pkl')

In [3]:
row = []
for mid,haha in ratings.items():
    for info in haha:
        info['movie'] = mid
        row.append(info)
df_ratings = pd.DataFrame(row)
row = []
for mid,haha in films.items():
    haha['movie'] = mid
    row.append(haha)
df_movie = pd.DataFrame(row)

In [4]:
df_movie

,title,director,actors,genre,movie
0,Toy Story (1995),[John Lasseter],"[Tom Hanks, Tim Allen, Don Rickles, Jim Varney...","[Adventure, Animation, Children, Comedy, Fantasy]",1
1,Jumanji (1995),[Joe Johnston],"[Robin Williams, Jonathan Hyde, Kirsten Dunst,...","[Adventure, Children, Fantasy]",2
2,Grumpier Old Men (1995),[Howard Deutch],"[Walter Matthau, Jack Lemmon, Ann-Margret, Sop...","[Comedy, Romance]",3
3,Waiting to Exhale (1995),[Forest Whitaker],"[Whitney Houston, Angela Bassett, Loretta Devi...","[Comedy, Drama, Romance]",4
4,Father of the Bride Part II (1995),[Charles Shyer],"[Steve Martin, Diane Keaton, Martin Short, Kim...",[Comedy],5
...,...,...,...,...,...
9546,Black Butler: Book of the Atlantic (2017),[Noriyuki Abe],"[Daisuke Ono, Maaya Sakamoto, Yukari Tamura, J...","[Action, Animation, Comedy, Fantasy]",193581
9547,No Game No Life: Zero (2017),[Atsuko Ishizuka],"[Youko Hikasa, Ai Kayano, Yoshitsugu Matsuoka,...","[Animation, Comedy, Fantasy]",193583
9548,Flint (2017),[Bruce Beresford],"[Queen Latifah, Betsy Brandt, Rob Morrow, Mari...",[Drama],193585
9549,Bungo Stray Dogs: Dead Apple (2018),[Takuya Igarashi],"[Kensho Ono, Mamoru Miyano, Kishou Taniyama, Y...","[Action, Animation]",193587


In [5]:
df_ratings

,user_rating,user_rating_date,user_id,movie
0,4.0,964982703,1,1
1,4.0,847434962,5,1
2,4.5,1106635946,7,1
3,2.5,1510577970,15,1
4,4.5,1305696483,17,1
...,...,...,...,...
100831,2.5,1479545749,610,160341
100832,4.5,1479544998,610,160527
100833,3.0,1493844794,610,160836
100834,3.5,1493848789,610,163937


In [6]:
final

NameError: name 'final' is not defined

In [ ]:
# a = df_movie['director'].value_counts()[df_movie['director'].value_counts() >= 2]
# df_movie = df_movie[df_movie['director'].isin(list(a.index))]
# df_movie.index = range(len(df_movie))

In [42]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [43]:
final = pd.merge(df_movie,df_ratings,on='movie')
a = final['user_id'].value_counts()[final['user_id'].value_counts() >= 10]
final = final[final['user_id'].isin(list(a.index))]
final.index = range(len(final))
hehe = final.copy()
del hehe['title']
del hehe['user_rating_date']

In [44]:
len(hehe['user_id'].value_counts())

610

In [45]:
hehe

,director,actors,genre,movie,user_rating,user_id
0,[John Lasseter],"[Tom Hanks, Tim Allen, Don Rickles, Jim Varney...","[Adventure, Animation, Children, Comedy, Fantasy]",1,4.0,1
1,[John Lasseter],"[Tom Hanks, Tim Allen, Don Rickles, Jim Varney...","[Adventure, Animation, Children, Comedy, Fantasy]",1,4.0,5
2,[John Lasseter],"[Tom Hanks, Tim Allen, Don Rickles, Jim Varney...","[Adventure, Animation, Children, Comedy, Fantasy]",1,4.5,7
3,[John Lasseter],"[Tom Hanks, Tim Allen, Don Rickles, Jim Varney...","[Adventure, Animation, Children, Comedy, Fantasy]",1,2.5,15
4,[John Lasseter],"[Tom Hanks, Tim Allen, Don Rickles, Jim Varney...","[Adventure, Animation, Children, Comedy, Fantasy]",1,4.5,17
...,...,...,...,...,...,...
100373,[Noriyuki Abe],"[Daisuke Ono, Maaya Sakamoto, Yukari Tamura, J...","[Action, Animation, Comedy, Fantasy]",193581,4.0,184
100374,[Atsuko Ishizuka],"[Youko Hikasa, Ai Kayano, Yoshitsugu Matsuoka,...","[Animation, Comedy, Fantasy]",193583,3.5,184
100375,[Bruce Beresford],"[Queen Latifah, Betsy Brandt, Rob Morrow, Mari...",[Drama],193585,3.5,184
100376,[Takuya Igarashi],"[Kensho Ono, Mamoru Miyano, Kishou Taniyama, Y...","[Action, Animation]",193587,3.5,184


In [46]:
movie = dict()
for i in range(df_movie.shape[0]):
    m = df_movie['movie'][i]
    movie.setdefault(m,dict())
    movie[m]['director'] = df_movie['director'][i]
    movie[m]['genre'] = df_movie['genre'][i]
    movie[m]['actors'] = df_movie['actors'][i]
    movie[m]['title'] = df_movie['title'][i]

In [47]:
def map_aspect_values_to_movies(x):
    (film, meta), aspect = x
    aspects = dict()
    if aspect == "director" and type(meta[aspect]) is str:
        aspects[meta[aspect]] = 1
    else:
        for g in meta[aspect]:
            aspects[g] = 1
    return film, meta, aspects


def dict_movie_aspect(paper_films, aspect):
    paper_films_aspect_prepended = map(
        lambda e: (e, aspect), list(paper_films.items()))
    aspect_dict = dict()
    with ProcessPoolExecutor(max_workers=THREADS) as executor:
        results = executor.map(map_aspect_values_to_movies,
                               paper_films_aspect_prepended)
    for film, meta, aspects in results:
        aspect_dict[film + "_" + meta["title"]] = aspects

    return aspect_dict

In [48]:
movie_genres = dict_movie_aspect(movie, "genre")
movie_directors = dict_movie_aspect(movie,'director')
movie_actors = dict_movie_aspect(movie,'actors')
actors = pd.DataFrame.from_dict(movie_actors,dtype = 'int64',orient = 'index').T.replace(np.nan, 0).index
directors = pd.DataFrame.from_dict(movie_directors,dtype = 'int64',orient = 'index').T.replace(np.nan, 0).index
genres = pd.DataFrame.from_dict(movie_genres,dtype = 'int64',orient = 'index').T.replace(np.nan, 0).index

<ipython-input-48-4af81307f982>:4: FutureWarning: In a future version, passing float-dtype values containing NaN and an integer dtype will raise IntCastingNaNError (subclass of ValueError) instead of silently ignoring the passed dtype. To retain the old behavior, call Series(arr) or DataFrame(arr) without passing a dtype.
  actors = pd.DataFrame.from_dict(movie_actors,dtype = 'int64',orient = 'index').T.replace(np.nan, 0).index
<ipython-input-48-4af81307f982>:5: FutureWarning: In a future version, passing float-dtype values containing NaN and an integer dtype will raise IntCastingNaNError (subclass of ValueError) instead of silently ignoring the passed dtype. To retain the old behavior, call Series(arr) or DataFrame(arr) without passing a dtype.
  directors = pd.DataFrame.from_dict(movie_directors,dtype = 'int64',orient = 'index').T.replace(np.nan, 0).index
<ipython-input-48-4af81307f982>:6: FutureWarning: In a future version, passing float-dtype values containing NaN and an integer dt

In [49]:
movie_genres

{'1_Toy Story (1995)': {'Adventure': 1,
  'Animation': 1,
  'Children': 1,
  'Comedy': 1,
  'Fantasy': 1},
 '2_Jumanji (1995)': {'Adventure': 1, 'Children': 1, 'Fantasy': 1},
 '3_Grumpier Old Men (1995)': {'Comedy': 1, 'Romance': 1},
 '4_Waiting to Exhale (1995)': {'Comedy': 1, 'Drama': 1, 'Romance': 1},
 '5_Father of the Bride Part II (1995)': {'Comedy': 1},
 '6_Heat (1995)': {'Action': 1, 'Crime': 1, 'Thriller': 1},
 '7_Sabrina (1995)': {'Comedy': 1, 'Romance': 1},
 '8_Tom and Huck (1995)': {'Adventure': 1, 'Children': 1},
 '9_Sudden Death (1995)': {'Action': 1},
 '10_GoldenEye (1995)': {'Action': 1, 'Adventure': 1, 'Thriller': 1},
 '11_American President, The (1995)': {'Comedy': 1, 'Drama': 1, 'Romance': 1},
 '12_Dracula: Dead and Loving It (1995)': {'Comedy': 1, 'Horror': 1},
 '13_Balto (1995)': {'Adventure': 1, 'Animation': 1, 'Children': 1},
 '14_Nixon (1995)': {'Drama': 1},
 '15_Cutthroat Island (1995)': {'Action': 1, 'Adventure': 1, 'Romance': 1},
 '16_Casino (1995)': {'Crime':

In [50]:
len(actors_dict)

18058

In [51]:
actors_dict = dict(zip(actors, range(len(actors))))
directors_dict = dict(zip(directors, range(
    len(actors), len(actors) + len(directors))))
genres_dict = dict(zip(genres, range(
    len(actors) + len(directors), len(actors) + len(directors) + len(genres))))
# max(actors_dict.values()),max(directors_dict.values()),max(genres_dict.values())

In [52]:
def change_director(x):
    qunima = itemgetter(*x)(directors_dict)
    if type(qunima) == int:
        return [qunima]
    else:
        return change_list(qunima)


def change_genre(x):
    qunima = itemgetter(*x)(genres_dict)
    if type(qunima) == int:
        return [qunima]
    else:
        return change_list(qunima)


def change_actor(x):
    qunima = itemgetter(*x)(actors_dict)
    if type(qunima) == int:
        return [qunima]
    else:
        return list(qunima)


def change_list(x):
    return list(x)

In [53]:
hehe = final.copy()
del hehe['title']
del hehe['user_rating_date']

In [54]:
hehe['director'] = hehe['director'].map(change_director)
hehe['actors'] = hehe['actors'].map(change_actor)
hehe['genre'] = hehe['genre'].map(change_genre)

In [55]:
hehe = hehe[['user_id','movie','actors','director','genre','user_rating']]
hehe['user_rating'] = hehe['user_rating'].astype('float')

In [56]:
hehe

,user_id,movie,actors,director,genre,user_rating
0,1,1,"[0, 1, 2, 3, 4]",[18058],"[22408, 22409, 22410, 22411, 22412]",4.0
1,5,1,"[0, 1, 2, 3, 4]",[18058],"[22408, 22409, 22410, 22411, 22412]",4.0
2,7,1,"[0, 1, 2, 3, 4]",[18058],"[22408, 22409, 22410, 22411, 22412]",4.5
3,15,1,"[0, 1, 2, 3, 4]",[18058],"[22408, 22409, 22410, 22411, 22412]",2.5
4,17,1,"[0, 1, 2, 3, 4]",[18058],"[22408, 22409, 22410, 22411, 22412]",4.5
...,...,...,...,...,...,...
100373,184,193581,"[18046, 14543, 17548, 18047, 17175]",[22404],"[22415, 22409, 22411, 22412]",4.0
100374,184,193583,"[18048, 18037, 17882, 18049, 18050]",[22405],"[22409, 22411, 22412]",3.5
100375,184,193585,"[2317, 18051, 1004, 15202, 18052]",[18552],[22414],3.5
100376,184,193587,"[17650, 17547, 18053, 18054, 17380]",[22406],"[22415, 22409]",3.5


In [57]:
le = LabelEncoder()
y = le.fit_transform(hehe['user_id'])
hehe['user_id'] = y

y = le.fit_transform(hehe['movie'])
hehe['movie'] = y


In [58]:
print("The number of aspects is {}".format(max(genres_dict.values()) + 1))
print("The number of users is {}".format(max(hehe['user_id'])+1))

The number of aspects is 22428
The number of users is 610


In [59]:
len(hehe['user_id'].value_counts()), len(hehe['movie'].value_counts())

(610, 9533)

In [60]:
hehe.shape

(100378, 6)

In [61]:
print("The number of items is {}".format(len(hehe['movie'].value_counts())))

The number of items is 9533


In [62]:
batch_size = 256
data = trainset
num_example = len(data)
indices = list(range(num_example))
indexs = indices[0:min(0+batch_size, num_example)]
users_index = data.iloc[:, 0].loc[indexs].values
user_id = torch.LongTensor(users_index).to(DEVICE)
actors_id = data.iloc[:, 2].loc[indexs]
actors_id.index = range(len(actors_id))
directors_id = data.iloc[:, 3].loc[indexs]
directors_id.index = range(len(directors_id))
genres_id = data.iloc[:, 4].loc[indexs]
genres_id.index = range(len(genres_id))
rating = torch.FloatTensor(
    data.iloc[:, 5].loc[indexs].values).to(DEVICE)
model = aspect_augumentation(610, 22428, 3, 8)
# prediction, scores, contribute_actors, contribute_directors, contribute_genres, cnm = model(
#     users, actors_id, directors_id, genres_id, batch_size)

In [63]:
users = model.user_factors(user_id)  # 128 * 8
aspects = model.entity_factors  # n_entity * 8
scores = torch.matmul(users, model.relation_k.weight)
scores = leaky(scores)
m = torch.nn.Softmax(dim=1)  # 128 * 3
scores = m(scores)
scores_actors = scores[:, 0]  # 128,
scores_directors = scores[:, 1]  # 128,
scores_genres = scores[:, 2]

In [64]:
x = directors_id
niubi = []
# users: 128 * 8
first = x.index[0]
val_base = x[first]
actor_base = torch.LongTensor(val_base)
actors_base = aspects(actor_base)  # 3 * 8
#actors_base = actors_base + items[first % 128].unsqueeze(1)
#actors_base = F.dropout(actors_base, p=rate, training=True)
pre_rating = torch.mm(actors_base, users[first].unsqueeze(1))
niubi.append(pre_rating)
pre_rating = pre_rating / pre_rating.shape[0]
pre_rating = pre_rating.sum(0)


In [65]:
directors_id[0:50]

0            [18404]
1            [18690]
2            [18225]
3            [19113]
4            [18902]
5            [18280]
6            [18187]
7            [18670]
8            [18722]
9            [19862]
10           [19635]
11           [18187]
12           [18408]
13           [18268]
14           [18160]
15           [21504]
16           [18167]
17           [18196]
18           [18490]
19           [20411]
20           [18083]
21           [18296]
22           [18125]
23           [18116]
24           [20411]
25           [18587]
26           [18193]
27           [18835]
28           [18615]
29           [18799]
30           [18111]
31           [18226]
32    [19646, 19647]
33           [18370]
34           [18427]
35           [18392]
36           [18323]
37           [18564]
38           [20452]
39           [19082]
40           [18338]
41           [18670]
42           [18074]
43           [19090]
44           [19166]
45           [18256]
46           [18368]
47           

In [66]:
x = genres_id
niubi = []
# users: 128 * 8
first = x.index[0]
val_base = x[first]
actor_base = torch.LongTensor(val_base)
actors_base = aspects(actor_base)  # 3 * 8
#actors_base = actors_base + items[first % 128].unsqueeze(1)
#actors_base = F.dropout(actors_base, p=rate, training=True)
pre_rating = torch.mm(actors_base, users[first].unsqueeze(1))
niubi.append(pre_rating)
pre_rating = pre_rating / pre_rating.shape[0]
pre_rating = pre_rating.sum(0)
for i in x.index[1:]:
    val = x[i]
    actor = torch.LongTensor(val)
    actors = aspects(actor)
    pre_ra = torch.mm(actors, users[i].unsqueeze(1))
    niubi.append(pre_ra)
    actors_f = pre_ra / pre_ra.shape[0]
    actors_f = actors_f.sum(0)
    pre_rating = torch.cat((pre_rating, actors_f))

In [67]:
pre_rating

tensor([-0.9556,  1.8830, -0.7722, -0.2030, -1.1125,  0.3724, -0.1278,  0.1485,
         0.0544, -0.0714, -0.0088,  0.2643, -2.3625, -0.4977, -0.4874,  1.5976,
        -0.7224,  0.0573,  2.1348,  0.8761,  0.7907, -0.0540,  0.1962, -0.1779,
        -2.1451, -1.4722,  0.5612,  1.0928, -3.0017, -3.1593, -0.2084,  4.4771,
        -0.3773, -1.0642, -1.9230,  1.7224, -2.4813, -1.6195,  0.3565,  0.9228,
         3.4122, -0.5074,  1.6855,  0.2136, -1.3282, -0.5427, -0.4912,  1.1821,
        -2.2516,  0.7714,  0.3277,  1.4589,  0.4979, -0.2864,  2.5222,  0.7708,
        -0.7645, -0.1568,  1.1821, -1.4112,  1.9716,  1.9450,  4.2570,  2.0034,
         0.5291, -1.3447,  0.0579, -0.1576,  4.2309, -0.6599, -0.1377, -2.7178,
        -1.7911,  0.9850, -0.5635, -2.5710,  1.1983,  0.9375,  0.6349,  1.9254,
        -1.7600,  3.6822,  1.8260, -1.7560, -0.3131, -1.2682, -2.5303, -0.8903,
        -1.7439, -0.8779, -0.4543, -3.4602,  1.6970,  1.5794,  0.0168,  1.1565,
         3.5963,  0.4700,  1.9126,  1.71

In [77]:
from tqdm import tqdm
from torch.nn import LeakyReLU
leaky = LeakyReLU(0.2)

def calculate_score(x, users, aspects):
    niubi = []
    # users: 128 * 8
    first = x.index[0]
    val_base = x[first]
    actor_base = torch.LongTensor(val_base)
    actors_base = aspects(actor_base)  # 3 * 8
    #actors_base = actors_base + items[first % 128].unsqueeze(1)
    #actors_base = F.dropout(actors_base, p=rate, training=True)
    pre_rating = torch.mm(actors_base, users[first].unsqueeze(1))
    niubi.append(pre_rating)
    pre_rating = pre_rating / pre_rating.shape[0]
    pre_rating = pre_rating.sum(0)
    for i in x.index[1:]:
        val = x[i]
        actor = torch.LongTensor(val)
        actors = aspects(actor)
        pre_ra = torch.mm(actors, users[i].unsqueeze(1))
        niubi.append(pre_ra)
        actors_f = pre_ra / pre_ra.shape[0]
        actors_f = actors_f.sum(0)
        pre_rating = torch.cat((pre_rating, actors_f))
    return pre_rating, niubi


class aspect_augumentation(nn.Module):
    def __init__(self, n_users, n_entity, n_rk, n_factors):
        super(aspect_augumentation, self).__init__()
        self.n_users = n_users
        self.n_entity = n_entity
        self.n_rk = n_rk
        self.n_factors = n_factors
        self.user_factors = torch.nn.Embedding(n_users, n_factors)
        self.entity_factors = torch.nn.Embedding(n_entity, n_factors)
        self.relation_k = torch.nn.Embedding(n_factors, n_rk)

    def forward(self, user_id, actors_id, directors_id, genres_id):
        '''
        user_factors: n_users * 64
        entity_factors: n_entity * 64
        relation_k: 64 * 3
        '''
        
        users = self.user_factors(user_id)  # 128 * 8
        aspects = self.entity_factors  # n_entity * 8
        users = F.dropout(users, p=rate, training=self.training)
        scores = torch.matmul(users, F.dropout(self.relation_k.weight, p=rate, training=self.training))  # 128 * 3
        scores = leaky(scores)
        m=torch.nn.Softmax(dim=1)  # 128 * 3
        scores=m(scores)  # 128 * 3

        '''
        Compute the importance of each aspects
        '''
        scores_actors=scores[:, 0] # 128,
        scores_directors=scores[:, 1] # 128,
        scores_genres=scores[:, 2] # 128,

        '''
        Compute the contribution of each aspects
        '''
        contribute_actors,niubi_act=calculate_score(
            actors_id, users,aspects)
        contribute_directors, niubi_dir=calculate_score(
            directors_id, users,aspects)
        contribute_genres, niubi_gen=calculate_score(
            genres_id, users, aspects)

        '''
        Compute the final predictions
        '''
        importance_sum=scores_actors + scores_directors + scores_genres
        prediction_sum=contribute_actors * scores_actors + contribute_directors * scores_directors + contribute_genres * scores_genres
        prediction=prediction_sum / importance_sum
        cnm = [niubi_act,niubi_dir,niubi_gen]

        return prediction, scores, contribute_actors, contribute_directors, contribute_genres,cnm

In [78]:
def binary_predictions(true_ratings, predicted_ratings):
    assert len(true_ratings) == len(predicted_ratings)
    binary_true_ratings = []
    binary_predicted_ratings = []

    for i in range(len(true_ratings)):
        if true_ratings[i] >= 3:
            binary_true_ratings.append(1)
        else:
            binary_true_ratings.append(0)

        if predicted_ratings[i] >= 3:
            binary_predicted_ratings.append(1)
        else:
            binary_predicted_ratings.append(0)

    return precision_score(binary_true_ratings, binary_predicted_ratings), recall_score(binary_true_ratings, binary_predicted_ratings), f1_score(binary_true_ratings, binary_predicted_ratings)

In [79]:
def arg_accuracy_int(ratings, predictions):
    ratings = ratings.cpu().detach().numpy()
    predictions = predictions.cpu().detach().numpy()
    total_nr = len(ratings)
    total_pred = 0
    for i in range(total_nr):
        (true_rating, pred_rating) = ratings[i], predictions[i]
        if round(pred_rating) >= int(true_rating)-1 and round(pred_rating) <= int(true_rating)+1:
            total_pred += 1

    return float(total_pred)/total_nr


def round_of_rating(number):
    return round(number * 2) / 2

In [80]:
def RMSE(data, model):
    users_index = data.iloc[:, 0].values
    users = torch.LongTensor(users_index).to(DEVICE)
    actors_id = data.iloc[:, 2]
    directors_id = data.iloc[:, 3]
    genres_id = data.iloc[:, 4]
    rating = torch.FloatTensor(
        data.iloc[:, 5].values).to(DEVICE)
    prediction,scores,contribute_actors,contribute_directors,contribute_genres,cnm = model(users,actors_id, directors_id, genres_id)
    rmse = loss_func(prediction, rating)
    mae = torch.nn.L1Loss()(prediction, rating)
    p,r,f = binary_predictions(rating, prediction)
    accuracy = arg_accuracy_int(rating,prediction)
    return rmse ** 0.5,mae,p,r,f, accuracy, cnm

In [81]:
def train(lr, dim, reg, batch_size, num_epochs, data, test):
    model = aspect_augumentation(610, 22428, 3, dim).to(DEVICE)
    optimizer = torch.optim.Adam(params=model.parameters(), lr=lr,weight_decay=reg)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='min', factor=0.5, patience=10, threshold_mode='abs',threshold = 0.005)
    for epoch in range(num_epochs):
        model.train()
        t1 = time.time()
        num_example = len(data)
        indices = list(range(num_example))
        for i in tqdm(range(0, num_example, batch_size)):
            optimizer.zero_grad()
            indexs = indices[i:min(i+batch_size, num_example)]
            users_index = data.iloc[:, 0].loc[indexs].values
            users = torch.LongTensor(users_index).to(DEVICE)
            actors_id = data.iloc[:, 2].loc[indexs]
            actors_id.index = range(len(actors_id))
            directors_id = data.iloc[:, 3].loc[indexs]
            directors_id.index = range(len(directors_id))
            genres_id = data.iloc[:, 4].loc[indexs]
            genres_id.index = range(len(genres_id))
            rating = torch.FloatTensor(
                data.iloc[:, 5].loc[indexs].values).to(DEVICE)
            prediction, scores, contribute_actors, contribute_directors, contribute_genres,cnm = model(
                users, actors_id, directors_id, genres_id)

            err = loss_func(prediction, rating) 
            err.backward()
            optimizer.step()
        t2 = time.time()
        rmse, mae, p, r, f, accuracy,cnm = RMSE(test, model)
        scheduler.step(rmse)
        print("Epoch: ", epoch, " Loss: ", err, " RMSE in test set:",
              rmse, "MAE in test set: ", mae)
        print("Accuracy in test set is: ", accuracy, "Precision in test set:",
              p, "Recall in test set: ", r, "F1 scores in test set is:", f)
        print("Time consumed is:", t2-t1)
    return rmse, mae, p, r, f, accuracy, model, cnm

In [82]:
trainset = hehe.sample(frac=0.8, random_state=0, axis=0)
trainset.index = range(len(trainset))
testset = hehe[~hehe.index.isin(trainset.index)]
testset.index = range(len(testset))
rate = 0.0

In [83]:
def grid(lrs, dims, regs, batch_sizes, num_epochs, data, test):
    res = []
    models = []
    qnm = []
    for lr in lrs:
        for dim in dims:
            for reg in regs:
                for batch_size in batch_sizes:
                    rmse, mae, p, r, f, accuracy, model,cnm = train(
                        lr, dim, reg, batch_size, num_epochs, data, test)
                    res.append(rmse)
                    res.append(mae)
                    res.append(p)
                    res.append(r)
                    res.append(f)
                    res.append(accuracy)
                    models.append(model)
                    state = { 'model': model.state_dict()}   
                    torch.save(state, '/home/jinfeng/Downloads/doctorant/Conférences/dsaa/argumrntative/KR2020-Aspect-Item-Recommender-System-master/data/MOVIELENS/ml-latest-small/models/model_' + str(lr) + '_' + str(dim) + '_' + str(reg) + '_' + str(batch_size) + '.pkl')
                    qnm.append(cnm)
    return res, models,qnm

In [84]:
from tqdm import tqdm
from torch.nn import LeakyReLU
leaky = LeakyReLU(0.2)

def calculate_score(x, users, aspects):
    niubi = []
    # users: 128 * 8
    first = x.index[0]
    val_base = x[first]
    actor_base = torch.LongTensor(val_base)
    actors_base = aspects(actor_base)  # 3 * 8
    #actors_base = actors_base + items[first % 128].unsqueeze(1)
    #actors_base = F.dropout(actors_base, p=rate, training=True)
    pre_rating = torch.mm(actors_base, users[first].unsqueeze(1))
    niubi.append(pre_rating)
    pre_rating = pre_rating / pre_rating.shape[0]
    pre_rating = pre_rating.sum(0)
    for i in x.index[1:]:
        val = x[i]
        actor = torch.LongTensor(val)
        actors = aspects(actor)
        pre_ra = torch.mm(actors, users[i].unsqueeze(1))
        niubi.append(pre_ra)
        actors_f = pre_ra / pre_ra.shape[0]
        actors_f = actors_f.sum(0)
        pre_rating = torch.cat((pre_rating, actors_f))
    return pre_rating, niubi


class aspect_augumentation(nn.Module):
    def __init__(self, n_users, n_entity, n_rk, n_factors):
        super(aspect_augumentation, self).__init__()
        self.n_users = n_users
        self.n_entity = n_entity
        self.n_rk = n_rk
        self.n_factors = n_factors
        self.user_factors = torch.nn.Embedding(n_users, n_factors)
        self.entity_factors = torch.nn.Embedding(n_entity, n_factors)
        self.relation_k = torch.nn.Embedding(n_factors, n_rk)
        

    def forward(self, user_id, actors_id, directors_id, genres_id):
        '''
        user_factors: n_users * 64
        entity_factors: n_entity * 64
        relation_k: 64 * 3
        '''
        
        users = self.user_factors(user_id)  # 128 * 8
        aspects = self.entity_factors  # n_entity * 8
        users = F.dropout(users, p=rate, training=self.training)
        #scores = torch.matmul(users, F.dropout(self.relation_k.weight, p=rate, training=self.training))  # 128 * 3
        #scores = leaky(scores)
        #m=torch.nn.Softmax(dim=1)  # 128 * 3
        #scores=m(scores)  # 128 * 3

        '''
        Compute the importance of each aspects
        '''
        #scores_actors=scores[:, 0] # 128,
        #scores_directors=scores[:, 1] # 128,
        #scores_genres=scores[:, 2] # 128,

        '''
        Compute the contribution of each aspects
        '''
        contribute_actors,niubi_act=calculate_score(
            actors_id, users,aspects)
        contribute_directors, niubi_dir=calculate_score(
            directors_id, users,aspects)
        contribute_genres, niubi_gen=calculate_score(
            genres_id, users, aspects)

        '''
        Compute the final predictions
        '''
        importance_sum = 1.0
        prediction_sum=contribute_actors * scores_actors + contribute_directors * scores_directors + contribute_genres * scores_genres
        prediction=prediction_sum / importance_sum
        cnm = [niubi_act,niubi_dir,niubi_gen]

        return prediction, scores, contribute_actors, contribute_directors, contribute_genres,cnm

In [86]:
trainset.to_csv('MOVIELENS/ml-latest-small/train.csv',index=False)
testset.to_csv('MOVIELENS/ml-latest-small/test.csv',index=False)

In [85]:
lrs = [1e-2]
dims = [32,16,8,4]
regs = [1e-6,4e-5,1e-7]
batch_sizes = [256]
num_epochs = 100
res_reg, models_reg,qnm = grid(lrs,dims,regs,batch_sizes,num_epochs,trainset,testset)

100%|██████████| 314/314 [03:29<00:00,  1.50it/s]


Epoch:  0  Loss:  tensor(3.0162, grad_fn=<MseLossBackward0>)  RMSE in test set: tensor(1.4996, grad_fn=<PowBackward0>) MAE in test set:  tensor(1.1692, grad_fn=<L1LossBackward0>)
Accuracy in test set is:  0.6836521219366407 Precision in test set: 0.860961745275772 Recall in test set:  0.6950266650130225 F1 scores in test set is: 0.769146307987922
Time consumed is: 209.44385719299316


100%|██████████| 314/314 [03:16<00:00,  1.59it/s]


Epoch:  1  Loss:  tensor(1.1460, grad_fn=<MseLossBackward0>)  RMSE in test set: tensor(1.0629, grad_fn=<PowBackward0>) MAE in test set:  tensor(0.8105, grad_fn=<L1LossBackward0>)
Accuracy in test set is:  0.8314405260011954 Precision in test set: 0.8946250265561929 Recall in test set:  0.783393277936252 F1 scores in test set is: 0.8353225113234372
Time consumed is: 196.90623712539673


100%|██████████| 314/314 [02:52<00:00,  1.82it/s]


...

Epoch:  97  Loss:  tensor(0.3239, grad_fn=<MseLossBackward0>)  RMSE in test set: tensor(0.6605, grad_fn=<PowBackward0>) MAE in test set:  tensor(0.4493, grad_fn=<L1LossBackward0>)
Accuracy in test set is:  0.9495915521020124 Precision in test set: 0.9394372603807915 Recall in test set:  0.8965025424779859 F1 scores in test set is: 0.9174678724416944
Time consumed is: 39.80275082588196


100%|██████████| 314/314 [00:39<00:00,  7.86it/s]


Epoch:  98  Loss:  tensor(0.3238, grad_fn=<MseLossBackward0>)  RMSE in test set: tensor(0.6605, grad_fn=<PowBackward0>) MAE in test set:  tensor(0.4493, grad_fn=<L1LossBackward0>)
Accuracy in test set is:  0.9495417413827456 Precision in test set: 0.9394687276742223 Recall in test set:  0.8969986357435198 F1 scores in test set is: 0.917742600640802
Time consumed is: 39.9756555557251


100%|██████████| 314/314 [00:38<00:00,  8.09it/s]


Epoch:  99  Loss:  tensor(0.3237, grad_fn=<MseLossBackward0>)  RMSE in test set: tensor(0.6605, grad_fn=<PowBackward0>) MAE in test set:  tensor(0.4493, grad_fn=<L1LossBackward0>)
Accuracy in test set is:  0.9495417413827456 Precision in test set: 0.9394451302709376 Recall in test set:  0.8966265657943694 F1 scores in test set is: 0.9175365675667101
Time consumed is: 38.80708932876587


In [91]:
#conding=utf8  
import os 

path = "/home/jinfeng/Downloads/doctorant/Conférences/dsaa/argumrntative/KR2020-Aspect-Item-Recommender-System-master/data/MOVIELENS/ml-latest-small/models" 
def get_file_name(path):
    for file in os.listdir(path):
        lr = file.split('_')[1]
        dim = file.split('_')[2]
        reg = file.split('_')[3]
        
        model = aspect_augumentation(610, 22428, 3, int(file.split('_')[2])).to(DEVICE)
        load_params = torch.load('/home/jinfeng/Downloads/doctorant/Conférences/dsaa/argumrntative/KR2020-Aspect-Item-Recommender-System-master/data/MOVIELENS/ml-latest-small/models/' + file)['model']
        model_params = model.state_dict()
        same_parsms = {k: v for k, v in load_params.items() if k in model_params.keys()}
        model_params.update(same_parsms)

        model.load_state_dict(model_params)
        rmse, mae, p, r, f, accuracy,cnm = RMSE(testset, model)
        print("Learning rate: ",lr, "dimension of vector: ", dim, "Regularization rate: ", reg)
        print(" RMSE in test set:",
              rmse, "MAE in test set: ", mae)
        print("Accuracy in test set is: ", accuracy, "Precision in test set:",
              p, "Recall in test set: ", r, "F1 scores in test set is:", f)

In [92]:
get_file_name(path)

Learning rate:  0.01 dimension of vector:  8 Regularization rate:  1e-06
 RMSE in test set: tensor(0.6033, grad_fn=<PowBackward0>) MAE in test set:  tensor(0.3928, grad_fn=<L1LossBackward0>)
Accuracy in test set is:  0.9605499103407054 Precision in test set: 0.9536628710590404 Recall in test set:  0.8984869155401215 F1 scores in test set is: 0.9252530412848431
Learning rate:  0.01 dimension of vector:  32 Regularization rate:  1e-06
 RMSE in test set: tensor(0.5354, grad_fn=<PowBackward0>) MAE in test set:  tensor(0.3221, grad_fn=<L1LossBackward0>)
Accuracy in test set is:  0.9664275752141861 Precision in test set: 0.9746524938675388 Recall in test set:  0.8870147587746496 F1 scores in test set is: 0.9287708590351277
Learning rate:  0.01 dimension of vector:  32 Regularization rate:  4e-05
 RMSE in test set: tensor(0.5540, grad_fn=<PowBackward0>) MAE in test set:  tensor(0.3980, grad_fn=<L1LossBackward0>)
Accuracy in test set is:  0.9675732217573222 Precision in test set: 0.95189466923

In [339]:
lrs = [1e-2]
dims = [8]
regs = [1e-6]
batch_sizes = [256]
num_epochs = 50
res_reg, models_reg,qnm = grid(lrs,dims,regs,batch_sizes,num_epochs,trainset,testset)

100%|██████████| 314/314 [00:45<00:00,  6.91it/s]


Epoch:  0  Loss:  tensor(2.3880, grad_fn=<AddBackward0>)  RMSE in test set: tensor(1.4120, grad_fn=<PowBackward0>) MAE in test set:  tensor(1.0499, grad_fn=<L1LossBackward0>)
Accuracy in test set is:  0.7525901574018729 Precision in test set: 0.8666990212731538 Recall in test set:  0.6644549175244946 F1 scores in test set is: 0.7522201551476009
Time consumed is: 45.47053122520447


100%|██████████| 314/314 [00:46<00:00,  6.77it/s]


Epoch:  1  Loss:  tensor(1.2317, grad_fn=<AddBackward0>)  RMSE in test set: tensor(1.0393, grad_fn=<PowBackward0>) MAE in test set:  tensor(0.7859, grad_fn=<L1LossBackward0>)
Accuracy in test set is:  0.8456365809922295 Precision in test set: 0.8743550224485693 Recall in test set:  0.8091281160858241 F1 scores in test set is: 0.8404779542014235
Time consumed is: 46.399190187454224


100%|██████████| 314/314 [00:49<00:00,  6.29it/s]


...

Epoch:  47  Loss:  tensor(0.3464, grad_fn=<AddBackward0>)  RMSE in test set: tensor(0.6866, grad_fn=<PowBackward0>) MAE in test set:  tensor(0.4477, grad_fn=<L1LossBackward0>)
Accuracy in test set is:  0.9427176728431959 Precision in test set: 0.9359365863166785 Recall in test set:  0.8932779362520153 F1 scores in test set is: 0.914109845480217
Time consumed is: 47.59498453140259


100%|██████████| 314/314 [00:46<00:00,  6.70it/s]


Epoch:  48  Loss:  tensor(0.3453, grad_fn=<AddBackward0>)  RMSE in test set: tensor(0.6864, grad_fn=<PowBackward0>) MAE in test set:  tensor(0.4474, grad_fn=<L1LossBackward0>)
Accuracy in test set is:  0.9428671050009962 Precision in test set: 0.9359890824018716 Recall in test set:  0.8931539129356318 F1 scores in test set is: 0.9140699371707812
Time consumed is: 46.89000201225281


100%|██████████| 314/314 [00:52<00:00,  6.04it/s]


Epoch:  49  Loss:  tensor(0.3444, grad_fn=<AddBackward0>)  RMSE in test set: tensor(0.6865, grad_fn=<PowBackward0>) MAE in test set:  tensor(0.4473, grad_fn=<L1LossBackward0>)
Accuracy in test set is:  0.942916915720263 Precision in test set: 0.9360264986685718 Recall in test set:  0.8937120178593576 F1 scores in test set is: 0.914379976525077
Time consumed is: 52.025819063186646


In [40]:
(1.09-0.61) / 1.09

0.4403669724770643

NameError: name 'DEVICE' is not defined

In [343]:
def xiba(hehe,model):
    users_index = hehe.iloc[:, 0].values
    users = torch.LongTensor(users_index)
    actors_id = hehe.iloc[:, 2]
    directors_id = hehe.iloc[:, 3]
    genres_id = hehe.iloc[:, 4]
    rating = torch.FloatTensor(
        hehe.iloc[:, 5].values)
    prediction,scores,contribute_actors,contribute_directors,contribute_genres, cnm = model(users, actors_id, directors_id, genres_id,12)
    return prediction,scores,contribute_actors,contribute_directors,contribute_genres,cnm

In [344]:
prediction, scores, contribute_actors, contribute_directors, contribute_genres,cnm = xiba(testset,models_reg[0])

In [345]:
rating = torch.FloatTensor(
        testset.iloc[:, 5].values)
rating

tensor([4.5000, 3.5000, 3.0000,  ..., 3.5000, 3.5000, 4.0000])

In [353]:
i = 0

In [354]:
prediction[i]

tensor(4.8094, grad_fn=<SelectBackward0>)

In [355]:
scores[i]

tensor([3.2654e-05, 2.9025e-01, 7.0971e-01], grad_fn=<SelectBackward0>)

In [356]:
contribute_actors[i]

tensor(-3.1903, grad_fn=<SelectBackward0>)

In [357]:
contribute_directors[i]

tensor(3.4051, grad_fn=<SelectBackward0>)

In [358]:
contribute_genres[i]

tensor(5.3841, grad_fn=<SelectBackward0>)

In [359]:
cnm[2][i]

tensor([[5.1657],
        [5.6025]], grad_fn=<MmBackward0>)

In [219]:
lrs = [5e-3]
dims = [8]
regs = [1e-7]
batch_sizes = [256]
num_epochs = 20
res_reg, models_reg = grid(lrs,dims,regs,batch_sizes,num_epochs,trainset,testset)

100%|██████████| 604/604 [01:32<00:00,  6.50it/s]


Epoch:  0  Loss:  tensor(3.7587, grad_fn=<AddBackward0>)  RMSE in test set: tensor(4.4199, grad_fn=<PowBackward0>) MAE in test set:  tensor(3.6052, grad_fn=<L1LossBackward0>)
Accuracy in test set is:  0.27592189765900144 Precision in test set: 0.8292256320559708 Recall in test set:  0.16503164556962024 F1 scores in test set is: 0.2752777851091346
Time consumed is: 92.9263973236084


100%|██████████| 604/604 [01:36<00:00,  6.29it/s]


Epoch:  1  Loss:  tensor(1.2809, grad_fn=<AddBackward0>)  RMSE in test set: tensor(4.7514, grad_fn=<PowBackward0>) MAE in test set:  tensor(3.7202, grad_fn=<L1LossBackward0>)
Accuracy in test set is:  0.36881085560389476 Precision in test set: 0.8286268972820332 Recall in test set:  0.2971518987341772 F1 scores in test set is: 0.43743594521568985
Time consumed is: 96.06445145606995


100%|██████████| 604/604 [01:49<00:00,  5.52it/s]


...

Epoch:  6  Loss:  tensor(0.8230, grad_fn=<AddBackward0>)  RMSE in test set: tensor(4.8327, grad_fn=<PowBackward0>) MAE in test set:  tensor(3.7618, grad_fn=<L1LossBackward0>)
Accuracy in test set is:  0.39064118500103584 Precision in test set: 0.8287833600978818 Recall in test set:  0.34297468354430377 F1 scores in test set is: 0.48517134095843495
Time consumed is: 94.91130757331848


100%|██████████| 604/604 [01:39<00:00,  6.08it/s]


Epoch:  7  Loss:  tensor(0.7409, grad_fn=<AddBackward0>)  RMSE in test set: tensor(4.8327, grad_fn=<PowBackward0>) MAE in test set:  tensor(3.7604, grad_fn=<L1LossBackward0>)
Accuracy in test set is:  0.3924539051170499 Precision in test set: 0.8297436675261641 Recall in test set:  0.3462341772151899 F1 scores in test set is: 0.4885901844326352
Time consumed is: 99.31716728210449


 77%|███████▋  | 465/604 [01:11<00:21,  6.47it/s]


KeyboardInterrupt: 

In [41]:
lrs = [1e-2]
dims = [8]
regs = [1e-7]
batch_sizes = [512]
num_epochs = 20
res_reg, models_reg = grid(lrs,dims,regs,batch_sizes,num_epochs,trainset,testset)

100%|██████████| 302/302 [01:40<00:00,  3.02it/s]


Epoch:  0  Loss:  tensor(7.8276, grad_fn=<AddBackward0>)  RMSE in test set: tensor(3.5700, grad_fn=<PowBackward0>) MAE in test set:  tensor(3.0696, grad_fn=<L1LossBackward0>)
Accuracy in test set is:  0.2170602858918583 Precision in test set: 0.8152125279642058 Recall in test set:  0.05765822784810127 F1 scores in test set is: 0.1076991281217674
Time consumed is: 100.10103178024292


100%|██████████| 302/302 [01:34<00:00,  3.21it/s]


Epoch:  1  Loss:  tensor(2.4813, grad_fn=<AddBackward0>)  RMSE in test set: tensor(2.4940, grad_fn=<PowBackward0>) MAE in test set:  tensor(1.7787, grad_fn=<L1LossBackward0>)
Accuracy in test set is:  0.5717837165941578 Precision in test set: 0.8238599871547848 Recall in test set:  0.487120253164557 F1 scores in test set is: 0.6122424628112322
Time consumed is: 94.22244501113892


100%|██████████| 302/302 [01:50<00:00,  2.72it/s]


...

Epoch:  17  Loss:  tensor(1.2225, grad_fn=<AddBackward0>)  RMSE in test set: tensor(1.1421, grad_fn=<PowBackward0>) MAE in test set:  tensor(0.9246, grad_fn=<L1LossBackward0>)
Accuracy in test set is:  0.7814895380153304 Precision in test set: 0.8191038111943869 Recall in test set:  0.812753164556962 F1 scores in test set is: 0.8159161305694544
Time consumed is: 110.55516982078552


100%|██████████| 302/302 [02:03<00:00,  2.45it/s]


Epoch:  18  Loss:  tensor(1.2187, grad_fn=<AddBackward0>)  RMSE in test set: tensor(1.1433, grad_fn=<PowBackward0>) MAE in test set:  tensor(0.9252, grad_fn=<L1LossBackward0>)
Accuracy in test set is:  0.7815931220219599 Precision in test set: 0.8191482548831251 Recall in test set:  0.8095569620253165 F1 scores in test set is: 0.8143243673404424
Time consumed is: 123.34250712394714


100%|██████████| 302/302 [01:38<00:00,  3.07it/s]


Epoch:  19  Loss:  tensor(1.2138, grad_fn=<AddBackward0>)  RMSE in test set: tensor(1.1445, grad_fn=<PowBackward0>) MAE in test set:  tensor(0.9265, grad_fn=<L1LossBackward0>)
Accuracy in test set is:  0.7810234099854982 Precision in test set: 0.8191794046661304 Recall in test set:  0.8055696202531646 F1 scores in test set is: 0.8123175109692861
Time consumed is: 98.28096842765808


In [36]:
lrs = [1e-1,5e-2,1e-2,5e-3,1e-4,5e-5,1e-5,5e-6]
dims = [8]
regs = [1e-7]
batch_sizes = [512]
num_epochs = 20
res_reg, models_reg = grid(lrs,dims,regs,batch_sizes,num_epochs,trainset,testset)

100%|██████████| 302/302 [01:37<00:00,  3.10it/s]


Epoch:  0  Loss:  tensor(1.5516, grad_fn=<AddBackward0>)  RMSE in test set: tensor(1.6168, grad_fn=<PowBackward0>) MAE in test set:  tensor(1.2388, grad_fn=<L1LossBackward0>)
Accuracy in test set is:  0.6776465713693806 Precision in test set: 0.8211525723117686 Recall in test set:  0.4707594936708861 F1 scores in test set is: 0.5984391342827259
Time consumed is: 97.29079699516296


100%|██████████| 302/302 [01:32<00:00,  3.28it/s]


Epoch:  1  Loss:  tensor(1.5962, grad_fn=<AddBackward0>)  RMSE in test set: tensor(1.3436, grad_fn=<PowBackward0>) MAE in test set:  tensor(1.0841, grad_fn=<L1LossBackward0>)
Accuracy in test set is:  0.7278848145846282 Precision in test set: 0.8237934385266762 Recall in test set:  0.5633860759493671 F1 scores in test set is: 0.6691473567495443
Time consumed is: 92.17915749549866


100%|██████████| 302/302 [01:32<00:00,  3.28it/s]


...

Epoch:  17  Loss:  tensor(13.1258, grad_fn=<AddBackward0>)  RMSE in test set: tensor(4.0497, grad_fn=<PowBackward0>) MAE in test set:  tensor(3.5410, grad_fn=<L1LossBackward0>)
Accuracy in test set is:  0.15514294592914854 Precision in test set: 0.8146591970121382 Recall in test set:  0.055221518987341775 F1 scores in test set is: 0.10343192460435068
Time consumed is: 88.16281938552856


100%|██████████| 302/302 [01:27<00:00,  3.47it/s]


Epoch:  18  Loss:  tensor(13.1242, grad_fn=<AddBackward0>)  RMSE in test set: tensor(4.0496, grad_fn=<PowBackward0>) MAE in test set:  tensor(3.5410, grad_fn=<L1LossBackward0>)
Accuracy in test set is:  0.1550652579241765 Precision in test set: 0.8145726296123307 Recall in test set:  0.05518987341772152 F1 scores in test set is: 0.10337571500548294
Time consumed is: 87.07614088058472


100%|██████████| 302/302 [01:28<00:00,  3.42it/s]


Epoch:  19  Loss:  tensor(13.1225, grad_fn=<AddBackward0>)  RMSE in test set: tensor(4.0495, grad_fn=<PowBackward0>) MAE in test set:  tensor(3.5409, grad_fn=<L1LossBackward0>)
Accuracy in test set is:  0.1550652579241765 Precision in test set: 0.8145726296123307 Recall in test set:  0.05518987341772152 F1 scores in test set is: 0.10337571500548294
Time consumed is: 88.31488370895386


In [37]:
res_reg

[tensor(1.5738, grad_fn=<PowBackward0>),
 tensor(1.1578, grad_fn=<L1LossBackward0>),
 0.8198829866270431,
 0.5587658227848101,
 0.6645965070761819,
 0.71801843795318,
 tensor(1.3180, grad_fn=<PowBackward0>),
 tensor(1.0472, grad_fn=<L1LossBackward0>),
 0.8175147109726549,
 0.7474050632911392,
 0.7808894032071416,
 0.7472809198259789,
 tensor(1.1159, grad_fn=<PowBackward0>),
 tensor(0.9091, grad_fn=<L1LossBackward0>),
 0.8196435248660102,
 0.8324050632911393,
 0.8259750047101677,
 0.7883001864512119,
 tensor(1.1330, grad_fn=<PowBackward0>),
 tensor(0.9190, grad_fn=<L1LossBackward0>),
 0.8197357959757813,
 0.8483227848101266,
 0.8337843301919069,
 0.7840273461777502,
 tensor(3.7191, grad_fn=<PowBackward0>),
 tensor(3.3913, grad_fn=<L1LossBackward0>),
 0.8199233716475096,
 0.013544303797468354,
 0.026648402963700888,
 0.11741247151439818,
 tensor(3.9119, grad_fn=<PowBackward0>),
 tensor(3.4978, grad_fn=<L1LossBackward0>),
 0.8040816326530612,
 0.031170886075949367,
 0.060015232292460016,


In [45]:
lrs = [5e-3]
dims = [8]
regs = [5e-5,1e-4,1e-5,5e-6,1e-6,5e-7,1e-7]
batch_sizes = [512]
num_epochs = 70
res_reg, models_reg = grid(lrs,dims,regs,batch_sizes,num_epochs,trainset,testset)

100%|██████████| 118/118 [00:36<00:00,  3.27it/s]


Epoch:  0  Loss:  tensor(12.2763, grad_fn=<AddBackward0>)  RMSE in test set: tensor(3.7818, grad_fn=<PowBackward0>) MAE in test set:  tensor(3.4629, grad_fn=<L1LossBackward0>)
Accuracy in test set is:  0.10532258692520344 Precision in test set: 0.7613636363636364 Recall in test set:  0.009606193844389219 F1 scores in test set is: 0.018973003586936002
Time consumed is: 36.105873346328735


100%|██████████| 118/118 [00:31<00:00,  3.74it/s]


Epoch:  1  Loss:  tensor(8.7868, grad_fn=<AddBackward0>)  RMSE in test set: tensor(3.7080, grad_fn=<PowBackward0>) MAE in test set:  tensor(3.4142, grad_fn=<L1LossBackward0>)
Accuracy in test set is:  0.10092278939376241 Precision in test set: 0.7481481481481481 Recall in test set:  0.0048269929267826415 F1 scores in test set is: 0.009592098390236952
Time consumed is: 31.581799268722534


100%|██████████| 118/118 [00:35<00:00,  3.34it/s]


...

Epoch:  57  Loss:  tensor(0.2479, grad_fn=<AddBackward0>)  RMSE in test set: tensor(1.1733, grad_fn=<PowBackward0>) MAE in test set:  tensor(0.9527, grad_fn=<L1LossBackward0>)
Accuracy in test set is:  0.7762333060779504 Precision in test set: 0.8143532635114233 Recall in test set:  0.8807111450965398 F1 scores in test set is: 0.846233324914472
Time consumed is: 34.06633019447327


100%|██████████| 118/118 [00:35<00:00,  3.36it/s]


Epoch:  58  Loss:  tensor(0.2402, grad_fn=<AddBackward0>)  RMSE in test set: tensor(1.1730, grad_fn=<PowBackward0>) MAE in test set:  tensor(0.9525, grad_fn=<L1LossBackward0>)
Accuracy in test set is:  0.777206712611455 Precision in test set: 0.8146202670168528 Recall in test set:  0.8894092907665838 F1 scores in test set is: 0.8503735520573923
Time consumed is: 35.116565227508545


 75%|███████▌  | 89/118 [00:25<00:08,  3.50it/s]


KeyboardInterrupt: 

In [48]:
lrs = [5e-3]
dims = [8]
regs = [5e-5]
batch_sizes = [256,512,1024,2048]
num_epochs = 70
res_batch, models_batch = grid(lrs,dims,regs,batch_sizes,num_epochs,trainset,testset)

100%|██████████| 268/268 [00:44<00:00,  6.08it/s]


Epoch:  0  Loss:  tensor(9.7096, grad_fn=<AddBackward0>)  RMSE in test set: tensor(3.8362, grad_fn=<PowBackward0>) MAE in test set:  tensor(3.4295, grad_fn=<L1LossBackward0>)
Accuracy in test set is:  0.13876883541642332 Precision in test set: 0.8595825426944972 Recall in test set:  0.03185429997890444 F1 scores in test set is: 0.06143205858421482
Time consumed is: 44.11094784736633


100%|██████████| 268/268 [00:40<00:00,  6.69it/s]


Epoch:  1  Loss:  tensor(6.4019, grad_fn=<AddBackward0>)  RMSE in test set: tensor(3.5561, grad_fn=<PowBackward0>) MAE in test set:  tensor(3.0254, grad_fn=<L1LossBackward0>)
Accuracy in test set is:  0.23577853054549702 Precision in test set: 0.8283582089552238 Recall in test set:  0.08585894100274243 F1 scores in test set is: 0.15559095253265373
Time consumed is: 40.04667043685913


100%|██████████| 268/268 [00:41<00:00,  6.43it/s]


...

Epoch:  67  Loss:  tensor(1.3540, grad_fn=<AddBackward0>)  RMSE in test set: tensor(1.3720, grad_fn=<PowBackward0>) MAE in test set:  tensor(1.1104, grad_fn=<L1LossBackward0>)
Accuracy in test set is:  0.705233033524121 Precision in test set: 0.8311460258780037 Recall in test set:  0.6323746571971028 F1 scores in test set is: 0.7182620502376101
Time consumed is: 40.848116636276245


100%|██████████| 34/34 [00:40<00:00,  1.20s/it]


Epoch:  68  Loss:  tensor(1.3502, grad_fn=<AddBackward0>)  RMSE in test set: tensor(1.3724, grad_fn=<PowBackward0>) MAE in test set:  tensor(1.1107, grad_fn=<L1LossBackward0>)
Accuracy in test set is:  0.7048242027800491 Precision in test set: 0.831233790292701 Recall in test set:  0.6310386048801069 F1 scores in test set is: 0.7174321461406243
Time consumed is: 40.71740126609802


100%|██████████| 34/34 [00:41<00:00,  1.22s/it]


Epoch:  69  Loss:  tensor(1.3447, grad_fn=<AddBackward0>)  RMSE in test set: tensor(1.3727, grad_fn=<PowBackward0>) MAE in test set:  tensor(1.1110, grad_fn=<L1LossBackward0>)
Accuracy in test set is:  0.7045321808199977 Precision in test set: 0.8313856427378965 Recall in test set:  0.630335419450109 F1 scores in test set is: 0.717033955925289
Time consumed is: 41.39833927154541


In [50]:
lrs = [5e-3]
dims = [8]
regs = [5e-5]
batch_sizes = [256,512,1024,2048]
num_epochs = 50
res_batch, models_batch = grid(lrs,dims,regs,batch_sizes,num_epochs,trainset,testset)

  0%|          | 0/67 [00:00<?, ?it/s]


KeyError: 'Passing list-likes to .loc or [] with any missing labels is no longer supported, see https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike'

In [32]:
rate = 0.0
lrs = [1e-1,5e-2,1e-2,5e-3,1e-4,5e-5,1e-5,5e-6]
dims = [16]
regs = [5e-5]
batch_sizes = [512]
num_epochs = 50
res, models = grid(lrs,dims,regs,batch_sizes,num_epochs,trainset,testset)

100%|██████████| 118/118 [00:35<00:00,  3.35it/s]


Epoch:  0  Loss:  tensor(2.7737, grad_fn=<AddBackward0>)  RMSE in test set: tensor(1.6088, grad_fn=<PowBackward0>) MAE in test set:  tensor(1.2990, grad_fn=<L1LossBackward0>)
Accuracy in test set is:  0.6341548884476113 Precision in test set: 0.817694780345936 Recall in test set:  0.2553049130185433 F1 scores in test set is: 0.38911752922751935
Time consumed is: 35.27388048171997


100%|██████████| 118/118 [00:30<00:00,  3.90it/s]


Epoch:  1  Loss:  tensor(1.7083, grad_fn=<AddBackward0>)  RMSE in test set: tensor(1.5866, grad_fn=<PowBackward0>) MAE in test set:  tensor(1.3043, grad_fn=<L1LossBackward0>)
Accuracy in test set is:  0.6250827395553479 Precision in test set: 0.819047619047619 Recall in test set:  0.250716880137641 F1 scores in test set is: 0.3839145230341396
Time consumed is: 30.272146463394165


100%|██████████| 118/118 [00:34<00:00,  3.43it/s]


...

Epoch:  1  Loss:  tensor(23.3385, grad_fn=<AddBackward0>)  RMSE in test set: tensor(4.3928, grad_fn=<PowBackward0>) MAE in test set:  tensor(3.7184, grad_fn=<L1LossBackward0>)
Accuracy in test set is:  0.18300042829887475 Precision in test set: 0.7954289996253279 Recall in test set:  0.10146243548078761 F1 scores in test set is: 0.1799686347645488
Time consumed is: 31.119622230529785


100%|██████████| 118/118 [00:33<00:00,  3.48it/s]


Epoch:  2  Loss:  tensor(23.3103, grad_fn=<AddBackward0>)  RMSE in test set: tensor(4.3916, grad_fn=<PowBackward0>) MAE in test set:  tensor(3.7177, grad_fn=<L1LossBackward0>)
Accuracy in test set is:  0.18303936456021494 Precision in test set: 0.7954204204204204 Recall in test set:  0.10127126744408335 F1 scores in test set is: 0.17966762760725796
Time consumed is: 33.922996520996094


 42%|████▏     | 50/118 [00:13<00:17,  3.79it/s]


KeyboardInterrupt: 

In [59]:
from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split
from surprise import Reader
from surprise.model_selection import cross_validate

reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(df_ratings[['user_id', 'movie', 'user_rating']], reader)


trainset, testset = train_test_split(data, test_size=0.2)


algo = SVD()

algo.fit(trainset)
predictions = algo.test(testset)

In [82]:
real = []
pre = []
for uid, iid, true_r, est, _ in predictions:
    real.append(true_r)
    pre.append(est)

In [83]:
real = torch.from_numpy(np.array(real))
pre = torch.from_numpy(np.array(pre))
rmse = loss_func(pre, real)
mae = torch.nn.L1Loss()(pre, real)
p, r, f = binary_predictions(real, pre)
accuracy = arg_accuracy_int(real, pre)

In [85]:
accuracy,p, r, f

(0.8795319836810099, 0.8610947458625676, 0.791273962152718, 0.8247092165839521)

In [86]:
rmse, mae

(tensor(0.9145, dtype=torch.float64), tensor(0.7551, dtype=torch.float64))